In [36]:
import requests
from lxml import html
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import queue

frontier = queue.Queue()
WEB_DRIVER_LOCATION = "C:/Users/Pirk/Desktop/faks-mag/ekstrakcija/chromedriver.exe"
TIMEOUT = 5
url =  "https://en.wikipedia.org/wiki/Main_Page"

def is_link(link):
    #logika - preveri če je stvar valid link ki ga damo v frontier
    
def make_absolute(baselink,link)
    #logika - prepozna če je link relativen inga spremeni v absolutnega

def crawl_site(url):

    WEB_PAGE_ADDRESS = url

    chrome_options = Options()
    chrome_options.add_argument("user-agent=fri-ieps-TEST")
    time.sleep(TIMEOUT)

    driver = webdriver.Chrome(WEB_DRIVER_LOCATION, options=chrome_options)
    driver.get(WEB_PAGE_ADDRESS)
    time.sleep(TIMEOUT)

    html = driver.page_source

    bsObj = BeautifulSoup(html, 'html.parser')
    links = bsObj.find_all('a', href=True)
    #finalLinks = set()
    for link in links:
        #finalLinks.add(link.attrs['href'])
        #klic metode, ki filtrira linke
        #klic metode, ki dopolni relativne linke v absolutne
        frontier.put(link.attrs['href'])


    

['#mw-head', '#searchInput', '/wiki/Wikipedia', '/wiki/Free_content', '/wiki/Encyclopedia', '/wiki/Help:Introduction_to_Wikipedia', '/wiki/Special:Statistics', '/wiki/English_language', '/wiki/Portal:The_arts', '/wiki/Portal:Biography', '/wiki/Portal:Geography', '/wiki/Portal:History', '/wiki/Portal:Mathematics', '/wiki/Portal:Science', '/wiki/Portal:Society', '/wiki/Portal:Technology', '/wiki/Wikipedia:Contents/Portals', '/wiki/File:Trial_of_Talat_Pasha.jpg', '/wiki/Assassination_of_Talaat_Pasha', '/wiki/Ottoman_Empire', '/wiki/Armenian_genocide', '/wiki/Soghomon_Tehlirian', '/wiki/Operation_Nemesis', '/wiki/Talaat_Pasha', '/wiki/Raphael_Lemkin', '/wiki/Genocide', '/wiki/International_law', '/wiki/Assassination_of_Talaat_Pasha', '/wiki/Group_(mathematics)', '/wiki/Farran_Zerbe', '/wiki/Fakhr_al-Din_II', '/wiki/Wikipedia:Today%27s_featured_article/March_2022', 'https://lists.wikimedia.org/postorius/lists/daily-article-l.lists.wikimedia.org/', '/wiki/Wikipedia:Featured_articles', '/wiki

{'https://e-uprava.gov.si/', 'http://www.izvoznookno.si', 'https://europa.eu/youreurope/index.htm#sl', 'http://www.gov.si', 'https://spot.gov.si/sl/kazalo/', 'http://www.gov.si/', 'https://evem.gov.si/evem/cert/uporabnik/prijava.evem', 'https://www.stopbirokraciji.gov.si/', 'https://spot.gov.si/sl/pomoc-in-podpora/tehnicna-podpora/', 'https://spot.gov.si/sl/e-postopki-in-storitve/pogoji-za-uporabo-e-postopkov-in-prijava-v-portal-spot/', 'https://spot.gov.si/sl/pomoc-in-podpora/', 'https://spot.gov.si/sl/', 'https://spot.gov.si/sl/portal-in-tocke-spot/', 'mailto:ekc@gov.si', 'https://spot.gov.si/sl/poslovanje/', 'https://www.spiritslovenia.si/', 'https://spot.gov.si/sl/e-postopki-in-storitve/pogoji-za-uporabo-e-postopkov-in-prijava-v-portal-spot/digitalna-potrdila/', 'https://spot.gov.si/sl/teme/pogoji-uporabe/', 'https://spot.gov.si/sl/dejavnosti-in-poklici/', 'https://spot.gov.si/sl/e-postopki-in-storitve/pooblascanje-oseb-za-opravljanje-postopkov-prek-portala-spot/', 'http://eugo.gov